https://www.kaggle.com/datasets/pwang001/user-package-information-of-mobile-operators

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#from tensorflow.keras.utils import to_categorical
#from keras import models
#from keras import layers

import warnings
warnings.filterwarnings("ignore")

In [82]:
df_train_1 = pd.read_csv('train_1.csv', encoding='utf-8')
df_train_2 = pd.read_csv('train_2.csv', encoding='utf-8')
df_test_1 = pd.read_csv('test_1.csv', encoding='utf-8')
df_test_2 = pd.read_csv('test_2.csv', encoding='utf-8')
df = pd.concat([df_train_1, df_train_2, df_test_1, df_test_2])

In [83]:
df.isnull().any()

service_type              False
is_mix_service            False
online_time               False
1_total_fee               False
2_total_fee               False
3_total_fee               False
4_total_fee               False
month_traffic             False
many_over_bill            False
contract_type             False
contract_time             False
is_promise_low_consume    False
net_service               False
pay_times                 False
pay_num                   False
last_month_traffic        False
local_trafffic_month      False
local_caller_time         False
service1_caller_time      False
service2_caller_time      False
gender                    False
age                       False
complaint_level           False
former_complaint_num      False
former_complaint_fee      False
current_service            True
user_id                   False
dtype: bool

Столбцы с нулами: current_service

In [85]:
allCs = df['current_service'].size
nullCs = df['current_service'][df['current_service'].isnull()].size
print('current_service count', allCs, sep=': ') 
print('current_service nulls count', nullCs, sep=': ')
print('percent of nulls', nullCs/allCs*100, sep=': ')

current_service count: 1479211
current_service nulls count: 360566
percent of nulls: 24.37556237751071


In [86]:
df['current_service'].unique()

array([99999825., 90063345., 90109916., 89950166., 89950168., 89950167.,
       90155946., 99999828., 99999826., 99999827., 99999830.,   999999.,
             nan])

In [87]:
# current_service - индексы планов а не сами планы, попробуем удалить колонку вместо удаления 25% строк
del df["current_service"]
df.isnull().any()

service_type              False
is_mix_service            False
online_time               False
1_total_fee               False
2_total_fee               False
3_total_fee               False
4_total_fee               False
month_traffic             False
many_over_bill            False
contract_type             False
contract_time             False
is_promise_low_consume    False
net_service               False
pay_times                 False
pay_num                   False
last_month_traffic        False
local_trafffic_month      False
local_caller_time         False
service1_caller_time      False
service2_caller_time      False
gender                    False
age                       False
complaint_level           False
former_complaint_num      False
former_complaint_fee      False
user_id                   False
dtype: bool

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1479211 entries, 0 to 160565
Data columns (total 26 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   service_type            1479211 non-null  int64  
 1   is_mix_service          1479211 non-null  int64  
 2   online_time             1479211 non-null  int64  
 3   1_total_fee             1479211 non-null  float64
 4   2_total_fee             1479211 non-null  object 
 5   3_total_fee             1479211 non-null  object 
 6   4_total_fee             1479211 non-null  float64
 7   month_traffic           1479211 non-null  float64
 8   many_over_bill          1479211 non-null  int64  
 9   contract_type           1479211 non-null  int64  
 10  contract_time           1479211 non-null  int64  
 11  is_promise_low_consume  1479211 non-null  int64  
 12  net_service             1479211 non-null  int64  
 13  pay_times               1479211 non-null  int64  
 14  pay

столбцы object: 
2_total_fee      
3_total_fee             
gender
age
user_id

In [89]:
# user_id ничего не определяет при классификации
del df["user_id"]

In [93]:
allCount = df['2_total_fee'].size
floatCount = df['2_total_fee'][df['2_total_fee'].apply(type) == float].size
print('2_total_fee count', allCount, sep=': ') 
print('2_total_fee float count', floatCount, sep=': ')
print('percent of nulls', floatCount/allCount*100, sep=': ')

2_total_fee count: 1479211
2_total_fee float count: 1118763
percent of nulls: 75.63241484818596


In [106]:
tf2 = pd.to_numeric(df['2_total_fee'], errors='coerce')

In [107]:
allCs = tf2.size
nullCs = tf2[tf2.isnull()].size
print('2_total_fee count', allCs, sep=': ') 
print('2_total_fee nulls count', nullCs, sep=': ')
print('percent of nulls', nullCs/allCs*100, sep=': ')

2_total_fee count: 1479211
2_total_fee nulls count: 15
percent of nulls: 0.0010140541139837386


Преобразовали всё что могли к типу float, непреобразованных - 15 штук << 1%. Можно удалить

In [108]:
df['2_total_fee'] = tf2
df1 = df[pd.notnull(df['2_total_fee'])]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1479196 entries, 0 to 160565
Data columns (total 25 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   service_type            1479196 non-null  int64  
 1   is_mix_service          1479196 non-null  int64  
 2   online_time             1479196 non-null  int64  
 3   1_total_fee             1479196 non-null  float64
 4   2_total_fee             1479196 non-null  float64
 5   3_total_fee             1479196 non-null  object 
 6   4_total_fee             1479196 non-null  float64
 7   month_traffic           1479196 non-null  float64
 8   many_over_bill          1479196 non-null  int64  
 9   contract_type           1479196 non-null  int64  
 10  contract_time           1479196 non-null  int64  
 11  is_promise_low_consume  1479196 non-null  int64  
 12  net_service             1479196 non-null  int64  
 13  pay_times               1479196 non-null  int64  
 14  pay

теперь 3_total_fee

In [109]:
tf3 = pd.to_numeric(df1['3_total_fee'], errors='coerce')
allCs = tf3.size
nullCs = tf3[tf3.isnull()].size
print('3_total_fee count', allCs, sep=': ') 
print('3_total_fee nulls count', nullCs, sep=': ')
print('percent of nulls', nullCs/allCs*100, sep=': ')

3_total_fee count: 1479196
3_total_fee nulls count: 5
percent of nulls: 0.0003380214657151588


In [112]:
df1['3_total_fee'] = tf3
df2 = df1[pd.notnull(df1['3_total_fee'])]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1479191 entries, 0 to 160565
Data columns (total 25 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   service_type            1479191 non-null  int64  
 1   is_mix_service          1479191 non-null  int64  
 2   online_time             1479191 non-null  int64  
 3   1_total_fee             1479191 non-null  float64
 4   2_total_fee             1479191 non-null  float64
 5   3_total_fee             1479191 non-null  float64
 6   4_total_fee             1479191 non-null  float64
 7   month_traffic           1479191 non-null  float64
 8   many_over_bill          1479191 non-null  int64  
 9   contract_type           1479191 non-null  int64  
 10  contract_time           1479191 non-null  int64  
 11  is_promise_low_consume  1479191 non-null  int64  
 12  net_service             1479191 non-null  int64  
 13  pay_times               1479191 non-null  int64  
 14  pay

теперь age

In [114]:
age = pd.to_numeric(df2['age'], errors='coerce')
allCs = age.size
nullCs = age[age.isnull()].size
print('age count', allCs, sep=': ') 
print('age nulls count', nullCs, sep=': ')
print('percent of nulls', nullCs/allCs*100, sep=': ')

age count: 1479191
age nulls count: 2
percent of nulls: 0.00013520904332165354


In [115]:
df2['age'] = age
df3 = df2[pd.notnull(df2['age'])]
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1479189 entries, 0 to 160565
Data columns (total 25 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   service_type            1479189 non-null  int64  
 1   is_mix_service          1479189 non-null  int64  
 2   online_time             1479189 non-null  int64  
 3   1_total_fee             1479189 non-null  float64
 4   2_total_fee             1479189 non-null  float64
 5   3_total_fee             1479189 non-null  float64
 6   4_total_fee             1479189 non-null  float64
 7   month_traffic           1479189 non-null  float64
 8   many_over_bill          1479189 non-null  int64  
 9   contract_type           1479189 non-null  int64  
 10  contract_time           1479189 non-null  int64  
 11  is_promise_low_consume  1479189 non-null  int64  
 12  net_service             1479189 non-null  int64  
 13  pay_times               1479189 non-null  int64  
 14  pay

факторизируем gender 

In [116]:
names = ['gender']
for name in names:
    df3[name] = pd.factorize(df3[name])[0]
df3 = df3.reset_index(drop=True)
df3.head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee
0,4,0,85,295.96,296.20,296.00,296.80,3813.614698,0,1,...,4096.000000,1392.038508,108.100000,0.000000,564.366667,0,31.0,0,0,0.0
1,1,0,10,265.20,261.20,208.50,174.50,0.000000,1,0,...,0.000000,62852.509718,240.100000,355.166667,0.000000,0,30.0,0,0,0.0
2,1,0,12,44.50,70.20,69.00,61.40,2598.397406,0,0,...,0.000000,1935.242104,27.666667,0.000000,98.416667,1,25.0,0,0,0.0
3,4,0,134,87.95,81.40,76.00,88.30,988.440563,0,0,...,37.336425,988.561075,89.900000,74.483333,121.833333,0,44.0,0,0,0.0
4,4,0,84,317.04,314.08,435.51,413.05,5885.800642,0,1,...,3305.741127,0.000000,0.000000,35.100000,496.733333,0,31.0,2,1,0.0


In [118]:
# размеры изначального и обработанного фреймов
print(df.shape[0])
print(df3.shape[0])

1479211
1479189


--------------------------------------------------------------------------------------------------------------------------
**************************************************************************************************************************
--------------------------------------------------------------------------------------------------------------------------

In [130]:
x = df3.loc[:, df.columns != 'gender']
y = df3['gender']
train_points, test_points, train_values, test_values = train_test_split(x, y, random_state=104, test_size=0.25, shuffle=False, stratify=None)

In [131]:
rf_model = ensemble.RandomForestClassifier(n_estimators=100)
rf_model.fit(train_points, train_values)
test_predict_rf = rf_model.predict(test_points)
print(accuracy_score(test_values, test_predict_rf))

0.8817678840880697
